Задача: обучить модель кредитного риск-менеджмента для банка.


Цель: оценить риск неуплаты клиента по кредиту (дефолт).

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# путь до данных на компьютере
#path = r"C:\Users\1\Documents\ИТОГОВЫЙ ПРОЕКТ_new\train_target.csv"
path = r"C:\Users\1\Documents\ИТОГОВЫЙ ПРОЕКТ_new\train_data\train_data"

In [ ]:
# для работающих в колабе
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# прописать свой путь
path = ''

In [ ]:
def reduce_mem_usage(df, int_cast=True, obj_to_category=True, subset=None):
    start_mem = df.memory_usage().sum() / 1024 ** 2;
    gc.collect()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    cols = subset if subset is not None else df.columns.tolist()

    for col in tqdm(cols):

        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            treat_as_int = str(col_type)[:3] == 'int'
            if treat_as_int:
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                    df[col] = df[col].astype(np.uint8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                    df[col] = df[col].astype(np.uint16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                    df[col] = df[col].astype(np.uint32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                    df[col] = df[col].astype(np.uint64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name and obj_to_category:
            df[col] = df[col].astype('category')
    gc.collect()
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.3f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
def read_parquet_dataset_from_local(path: str, file_name_tag: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    res = []
    dataset_paths = sorted([os.path.join(path, filename) for filename in os.listdir(path)
                              if filename.startswith(file_name_tag)])

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        chunk = reduce_mem_usage(chunk)
        chunk['rn'] = 1 # Добавляем столбец 'rn' здесь!
        res.append(chunk)
        del chunk
        gc.collect()

    return pd.concat(res).fillna(0).reset_index(drop=True)

In [ ]:
print("Найденные файлы:", path) # Добавьте эту строку для вывода dataset_paths

In [ ]:
def prepare_transactions_dataset(path: str, file_name_tag: str, num_parts_to_preprocess_at_once: int = 1, 
                               num_parts_total: int = 50, save_to_path=None, verbose: bool = False):
    """
    Возвращает готовый pd.DataFrame с признаками, на которых можно обучать модель.
    """
    preprocessed_frames = []

    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                     desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path, file_name_tag, step, 
                                                           num_parts_to_preprocess_at_once,
                                                           verbose=verbose)
        
        # Препроцессинг данных
        prepared_df = transactions_frame[['id', 'rn']].copy()
        features = [f for f in transactions_frame.columns if f not in ['id', 'rn']]
        dummies = pd.get_dummies(transactions_frame[features], columns=features, dtype='int8')
        prepared_df = pd.concat([prepared_df, dummies], axis=1)
        
        # Группировка и агрегация
        agg_d = {f: 'sum' for f in prepared_df.columns if f not in ['id', 'rn']}
        agg_d['rn'] = 'count'
        prepared_df = prepared_df.groupby('id').agg(agg_d).astype(int).reset_index()
        
        if save_to_path:
            block_as_str = str(step).zfill(3)
            prepared_df.to_parquet(os.path.join(save_to_path, f'processed_chunk_{block_as_str}.parquet'))
        else:
            preprocessed_frames.append(prepared_df)
        
        gc.collect()
    
    return pd.concat(preprocessed_frames, ignore_index=True) if not save_to_path else None
# Пример вызова
data = prepare_transactions_dataset(
    path, 
    'train', 
    num_parts_to_preprocess_at_once=1, 
    num_parts_total=12,
    save_to_path=r"C:\Users\1\Documents\ИТОГОВЫЙ ПРОЕКТ_new\processed_data"
)


        

In [ ]:
data = read_parquet_dataset_from_local(r"C:\Users\1\Documents\ИТОГОВЫЙ ПРОЕКТ_new\processed_data", 'processed', 0, 12, verbose=1)


In [ ]:
# пример полученных данны
data.head()

In [ ]:
data.shape

In [ ]:
# объем данных
data.info(max_cols=5, memory_usage='deep')

In [ ]:
# добавим значения целевой переменной
target_path = r"C:\Users\1\Documents\ИТОГОВЫЙ ПРОЕКТ_new\train_target.csv"
targets = pd.read_csv(target_path)

# --- ОТЛАДКА: Проверьте структуру targets DataFrame ---
print("--- Проверка targets DataFrame ---")
print("Столбцы в targets DataFrame:", targets.columns)
print("Первые 5 строк targets DataFrame:\n", targets.head())
print("----------------------------------\n")
# --- Конец отладочного блока ---

# Установка 'id' в качестве индекса для обоих DataFrame
# Теперь мы знаем, что 'id' в 'data' - это колонка, а не индекс, до set_index.
# Если 'id' по какой-то причине уже был бы индексом, этот шаг ничего не изменит,
# но он обязателен, если 'id' - это колонка с RangeIndex.
print("Попытка установить 'id' в качестве индекса для 'data'...")
if 'id' in data.columns:
    data = data.set_index('id')
    print("'id' успешно установлен как индекс для 'data'.")
else:
    print("Предупреждение: Колонка 'id' не найдена в 'data'. Возможно, она уже является индексом или отсутствует.")

print("Попытка установить 'id' в качестве индекса для 'targets'...")
if 'id' in targets.columns:
    targets = targets.set_index('id')
    print("'id' успешно установлен как индекс для 'targets'.")
else:
    print("Ошибка: Колонка 'id' не найдена в файле train_target.csv. Невозможно установить индекс.")
    # Здесь можно добавить sys.exit() или raise Error, если это критично.



print("Объединение 'data' и 'targets' по 'id' индексу...")
data = data.merge(targets[['flag']], left_index=True, right_index=True, how='left')

# Переименовываем столбец 'flag' в 'target' для единообразия, если нужно
data.rename(columns={'flag': 'target'}, inplace=True)
print("Столбец 'flag' переименован в 'target'.")

# Сброс индекса 'id' обратно в колонку (если нужно для дальнейшей работы)
# ИСПРАВЛЕНИЕ ОПЕЧАТКИ: Было 'all_data', теперь 'data'
print("Сброс индекса обратно в колонку...")
data = data.reset_index()
print("Индекс сброшен.")


print("\n--- Результат после обработки ---")
data.head()
data.info(max_cols=5, memory_usage='deep')

In [ ]:
train_data_target = data.merge(targets, on="id")

In [ ]:
train_data_target.shape, data.shape

In [ ]:
del data
gc.collect()

In [ ]:
# убедимся что нет нанов
train_data_target.isna().sum().sum()

In [ ]:
train_data_target.columns

In [ ]:
# подготовим фичи и таргет
train_data = train_data_target.drop(['flag', 'id'], axis=1)
train_labels = train_data_target['flag']
del train_data_target

In [ ]:
# посмотрим что нет колонок с 1 значением
train_data.nunique()

In [ ]:
train_data.head()

In [ ]:
# катбуст
!pip install catboost

In [ ]:
from catboost import cv
from catboost import CatBoostClassifier

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import time # Для измерения времени обучения
# from tqdm.notebook import tqdm # Если вы в Jupyter Notebook, используйте эту tqdm
# Если вы не в Jupyter Notebook, можете использовать просто from tqdm import tqdm
# Но для этого скрипта tqdm не нужна, так как CatBoost сам показывает прогресс.

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
# Удаляем 'id', если он не является признаком, а просто идентификатором.
# Убедитесь, что 'id' действительно есть в колонках, если пытаетесь его удалить.
features_to_drop = ['target']
if 'id' in train_data.columns:
    features_to_drop.append('id')

X = train_data.drop(columns=features_to_drop)
y = train_data['target']

print(f"Размер X (признаков): {X.shape}")
print(f"Размер y (целевой переменной): {y.shape}")

In [ ]:
# Разделение данных на обучающую и тестовую выборки
# stratify=y используется для сохранения пропорций классов в обеих выборках
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
# подбор через перебор параметров, оставляем тут нужный, как пример
tree_params = {
    "objective": "Logloss",
    "eval_metric": "AUC",
    "l2_leaf_reg": 0.5,
    "learning_rate": 0.6,
    "n_estimators": 1000,
    "early_stopping_rounds": 50
}

In [ ]:
# --- 2. Обучение модели CatBoost и измерение времени ---
print("\nНачинаем обучение модели CatBoost с заданными параметрами...")
print(f"Параметры обучения: {tree_params}")

model = CatBoostClassifier(**tree_params)

In [ ]:
start_time = time.time() # Записываем время начала обучения

In [ ]:
model.fit(X_train, y_train,
          eval_set=(X_test, y_test), # Набор данных для отслеживания early stopping и метрик
          # Если у вас есть категориальные признаки, которые не были One-Hot encoded,
          # их нужно указать CatBoost'у. Например:
          # cat_features=['categorical_col1', 'categorical_col2'],
          plot=False # Установите в True, если хотите видеть живой график обучения в Jupyter Notebook
         )

end_time = time.time() # Записываем время окончания обучения
training_time = end_time - start_time

print(f"\nОбучение модели завершено. Время обучения: {training_time:.2f} секунд.")

In [ ]:
# --- 3. Прогнозирование на тестовой выборке ---
# Вероятности для ROC AUC
y_pred_proba = model.predict_proba(X_test)[:, 1]
# Предсказанные классы для матрицы ошибок
y_pred = model.predict(X_test)


In [ ]:
# --- 4. Построение ROC AUC кривой ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(9, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
print(f"Значение ROC AUC на тестовой выборке: {roc_auc:.4f}")

# --- 5. Построение матрицы ошибок (Confusion Matrix) ---
cm = confusion_matrix(y_test, y_pred)
# Создаем объект ConfusionMatrixDisplay для удобного отображения
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

plt.figure(figsize=(8, 8))
# Отображаем матрицу ошибок на текущих осях
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title('Матрица ошибок (Confusion Matrix)')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.grid(False) # Отключаем сетку для матрицы
plt.show()

print("\nМатрица ошибок (Confusion Matrix):\n", cm)

In [ ]:
#БУДЕМ ИСПОЛЬЗОАТЬ ПОДБОР ПО СЕТКЕ

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
# Удаляем 'id', если он не является признаком, а просто идентификатором.
features_to_drop = ['target']
if 'id' in train_data.columns:
    features_to_drop.append('id')

X = train_data.drop(columns=features_to_drop)
y = train_data['target']

print(f"Размер X (признаков): {X.shape}")
print(f"Размер y (целевой переменной): {y.shape}")

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}, y_test: {y_test.shape}")


In [ ]:
# --- 1. Параметры модели CatBoost (базовые, не те, что подбираются) ---
base_tree_params = {
    "objective": "Logloss",
    "eval_metric": "AUC",
    "l2_leaf_reg": 0.5,
    "learning_rate": 0.6,
    "n_estimators": 1000,
    "early_stopping_rounds": 50
}

In [ ]:
# --- 2. Определение сетки для подбора гиперпараметров ---
grid = {
    'l2_leaf_reg': [0.01, 0.1, 1, 10, 100, 110]
}


In [ ]:
# --- 3. Инициализация модели и запуск grid_search ---
print("\nНачинаем подбор гиперпараметров CatBoost с помощью grid_search...")
print(f"Базовые параметры: {base_tree_params}")
print(f"Параметры для подбора (сетка): {grid}")

# Создаем экземпляр CatBoostClassifier с базовыми параметрами
model = CatBoostClassifier(**base_tree_params)

start_time = time.time() # Записываем время начала подбора


In [ ]:
# Запускаем grid_search
# cv - количество фолдов для кросс-валидации для каждого набора параметров
# refit=True - после завершения поиска, модель автоматически переобучится на всем X_train
#              с лучшими найденными параметрами. Это удобно, т.к. model_grid_search
#              сразу станет вашей обученной моделью.
# verbose=True - выводить информацию о ходе grid_search
model.grid_search(grid,
                  X=X_train,
                  y=y_train,
                  cv=3, # Рекомендуется использовать 3 или 5 фолдов для CV
                  refit=True,
                  plot=False, # Установите в True, если хотите видеть живой график в Jupyter Notebook
                  verbose=True) # Показывает прогресс поиска

end_time = time.time() # Записываем время окончания подбора
training_time = end_time - start_time

print(f"\nПодбор гиперпараметров и обучение лучшей модели завершено. Общее время: {training_time:.2f} секунд.")


In [ ]:
# --- 4. Проверка лучших найденных параметров ---
print("\nЛучшие найденные параметры:")
print(model.get_params())

In [ ]:
# --- 5. Прогнозирование на тестовой выборке (с лучшей моделью) ---
print("\nВыполняем предсказания на тестовой выборке...")
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

In [ ]:
# --- 6. Построение ROC AUC кривой ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(9, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve (After Grid Search)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"Значение ROC AUC на тестовой выборке: {roc_auc:.4f}")


In [ ]:
# --- 7. Построение матрицы ошибок (Confusion Matrix) ---
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

plt.figure(figsize=(8, 8))
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title('Матрица ошибок (Confusion Matrix) (After Grid Search)')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.grid(False)
plt.show()

print("\nМатрица ошибок (Confusion Matrix):\n", cm)

In [ ]:
# ПОСТРОИМ МОДЕЛЬ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler # Для масштабирования признаков
features_to_drop = ['target']
if 'id' in train_data.columns:
    features_to_drop.append('id')

X = train_data.drop(columns=features_to_drop)
y = train_data['target']

# --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ 1: Приведение всех признаков к float32 ---
print(f"Исходный тип данных признаков: {X.dtypes.iloc[0]}")
X = X.astype(np.float32)
print(f"Новый тип данных признаков: {X.dtypes.iloc[0]} (уменьшено для экономии памяти)")

print(f"Размер X (признаков): {X.shape}")
print(f"Размер y (целевой переменной): {y.shape}")

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}, y_test: {y_test.shape}")


In [ ]:
# --- 2. Масштабирование признаков ---
print("\nВыполняем масштабирование признаков (StandardScaler)...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Масштабирование завершено.")


In [ ]:
# Преобразуем обратно в DataFrame, чтобы сохранить имена колонок (необязательно, но полезно для отладки)
# и явно убедимся, что тип данных float32
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index).astype(np.float32)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index).astype(np.float32)
print(f"Тип данных масштабированных признаков: {X_train_scaled.dtypes.iloc[0]}")

In [ ]:
# --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ 2: Создание подвыборки для GridSearchCV ---
# Если у вас 3М записей, `GridSearchCV` на всех данных будет очень медленным и может
# все равно вызвать MemoryError из-за множества копий.
# Обучение на подвыборке для поиска лучших параметров - это стандартная практика.
# Финальную модель можно потом обучить на всем X_train_scaled.
subset_ratio = 0.1 # Использовать 10% обучающих данных для Grid Search
# Вы можете настроить этот процент в зависимости от доступной памяти и желаемой точности подбора.
# Если 0.1 все еще вызывает ошибку, попробуйте 0.05 или 0.02.

print(f"\nСоздаем подвыборку ({subset_ratio*100:.0f}%) обучающих данных для Grid Search...")
X_train_subset, _, y_train_subset, _ = train_test_split(
    X_train_scaled, y_train, test_size=(1-subset_ratio), random_state=42, stratify=y_train
)
print(f"Размер X_train_subset для Grid Search: {X_train_subset.shape}")

In [ ]:
# --- 2. Определение сетки для подбора гиперпараметров ---
# Для логистической регрессии с большим объемом данных:
# C: Инверсия силы регуляризации. Меньше C -> сильнее регуляризация.
# penalty: 'l1', 'l2', 'elasticnet', 'none'
# solver: Алгоритм оптимизации.
#   'liblinear' - хороший выбор для небольших данных, поддерживает L1/L2.
#   'saga' - поддерживает L1, L2, elasticnet и 'none'. Хорошо для больших данных.
#   'lbfgs' - по умолчанию, не поддерживает L1.
#   'newton-cg', 'sag' - поддерживают L2 и 'none'.
#   'newton-cholesky' - L2, 'none'.
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0], # Пример значений C. Можно добавить 0.01, 100.0 и т.д.
    'penalty': ['l2'],    # Начнем с L2. Для 'l1' или 'elasticnet' нужен 'solver'='liblinear' или 'saga'
    'solver': ['saga'],   # 'saga' - хороший универсальный выбор для больших данных
    'max_iter': [1000],
    'class_weight': [None, 'balanced'] # Добавляем этот параметр# Увеличим количество итераций, чтобы модель успела сойтись
}
print(f"\nСетка параметров для подбора: {param_grid}")

In [ ]:
# Если вы хотите попробовать L1 регуляризацию, можно сделать так (но это может увеличить время):
# param_grid = [
#     {
#         'C': [0.1, 1.0, 10.0],
#         'penalty': ['l2'],
#         'solver': ['lbfgs', 'saga'], # lbfgs не поддерживает l1
#         'max_iter': [1000]
#     },
#     {
#         'C': [0.1, 1.0, 10.0],
#         'penalty': ['l1'],
#         'solver': ['liblinear', 'saga'], # Эти солверы поддерживают l1
#         'max_iter': [1000]
#     }
# ]


In [ ]:
# --- 3. Инициализация LogisticRegression и GridSearchCV ---
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler # Для масштабирования признаков
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings # Для управления предупреждениями
warnings.filterwarnings('ignore', category=UserWarning) # Скрываем предупреждения от sklearn.linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
print("\nНачинаем подбор гиперпараметров логистической регрессии с помощью GridSearchCV...")
print(f"Сетка параметров для подбора: {param_grid}")

# Инициализируем модель логистической регрессии
# random_state для воспроизводимости
# n_jobs=-1 - использовать все доступные ядра CPU для параллелизации обучения
#           (GridSearchCV сам параллелизует по фолдам и параметрам)
log_reg = LogisticRegression(random_state=42, n_jobs=-1)

# Инициализируем GridSearchCV
# cv=3 - количество фолдов для кросс-валидации
# scoring='roc_auc' - метрика, по которой будет выбираться лучшая модель
# verbose=3 - уровень детализации вывода
# n_jobs=-1 - использовать все ядра CPU для параллелизации GridSearchCV
grid_search = GridSearchCV(estimator=log_reg,
                           param_grid=param_grid,
                           cv=3,
                           scoring='roc_auc',
                           verbose=3,
                           n_jobs=-1)

start_time = time.time() # Записываем время начала подбора

In [ ]:
# Запускаем поиск по сетке на МАШТАБИРОВАННОЙ ПОДВЫБОРКЕ данных
grid_search.fit(X_train_subset, y_train_subset)

end_time = time.time()
tuning_time = end_time - start_time

print(f"\nПодбор гиперпараметров логистической регрессии завершен. Время подбора: {tuning_time:.2f} секунд.")

In [ ]:
# --- 4. Проверка лучших найденных параметров и лучшей модели ---
print("\nЛучшие найденные параметры:")
print(grid_search.best_params_)
print(f"Лучший ROC AUC на кросс-валидации: {grid_search.best_score_:.4f}")

# Лучшая модель доступна через .best_estimator_
best_log_reg_model = grid_search.best_estimator_
print(f"Лучшая модель: {best_log_reg_model}")

In [ ]:
# --- 5. Обучение финальной модели на всем масштабированном X_train ---
print("\nОбучаем финальную модель логистической регрессии на ВСЕМ масштабированном обучающем наборе...")
best_params = grid_search.best_params_
print(f"Лучшие найденные параметры: {best_params}")

final_log_reg_model = LogisticRegression(random_state=42, n_jobs=-1, **best_params)

start_time_final_train = time.time()
final_log_reg_model.fit(X_train_scaled, y_train) # Обучаем на всем X_train_scaled (float32)
end_time_final_train = time.time()
final_training_time = end_time_final_train - start_time_final_train

print(f"Финальная модель обучена на полном наборе данных. Время обучения: {final_training_time:.2f} секунд.")


In [ ]:
# --- 6. Прогнозирование на тестовой выборке (с лучшей моделью) ---
print("\nВыполняем предсказания на масштабированной тестовой выборке...")
y_pred_proba = final_log_reg_model.predict_proba(X_test_scaled)[:, 1]
y_pred = final_log_reg_model.predict(X_test_scaled)



In [ ]:
# --- 7. Построение ROC AUC кривой ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(9, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve (Logistic Regression - Final Model)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"Значение ROC AUC на тестовой выборке: {roc_auc:.4f}")

In [ ]:
# --- 8. Построение матрицы ошибок (Confusion Matrix) ---
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=final_log_reg_model.classes_)

plt.figure(figsize=(8, 8))
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title('Матрица ошибок (Confusion Matrix) (Logistic Regression - Final Model)')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.grid(False)
plt.show()

print("\nМатрица ошибок (Confusion Matrix):\n", cm)

In [ ]:
# ПОСТРОИМ МОДЕЛЬ LightGBM

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV # Будем использовать GridSearchCV для примера, но RandomizedSearchCV также хорош.
from lightgbm import LGBMClassifier # Импортируем LightGBM
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
features_to_drop = ['target']
if 'id' in train_data.columns:
    features_to_drop.append('id')

X = train_data.drop(columns=features_to_drop)
y = train_data['target']

In [ ]:
# --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ 1: Приведение всех признаков к float32 ---
print(f"Исходный тип данных признаков: {X.dtypes.iloc[0]}")
X = X.astype(np.float32)
print(f"Новый тип данных признаков: {X.dtypes.iloc[0]} (уменьшено для экономии памяти)")

print(f"Размер X (признаков): {X.shape}")
print(f"Размер y (целевой переменной): {y.shape}")

# Разделение данных на обучающую и тестовую выборки
# LightGBM не требует масштабирования, но работаем с float32
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
# --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ 2: Создание подвыборки для GridSearchCV ---
# Для подбора гиперпараметров LightGBM на больших данных используем подвыборку.
subset_ratio = 0.1 # Использовать 10% обучающих данных для Grid Search

print(f"\nСоздаем подвыборку ({subset_ratio*100:.0f}%) обучающих данных для Grid Search...")
X_train_subset, _, y_train_subset, _ = train_test_split(
    X_train, y_train, test_size=(1-subset_ratio), random_state=42, stratify=y_train
)
print(f"Размер X_train_subset для Grid Search: {X_train_subset.shape}")


# --- 3. Определение сетки для подбора гиперпараметров LightGBM ---
# Это лишь пример, для лучших результатов потребуется более тонкая настройка.
# Обратите внимание на `early_stopping_round`, он будет использоваться во время fit.
param_grid = {
    'n_estimators': [300, 500], # Количество деревьев. LightGBM может работать с большим числом, если есть early_stopping.
    'learning_rate': [0.05, 0.1], # Скорость обучения. Меньше learning_rate -> больше деревьев, но может дать лучшую точность.
    'num_leaves': [20, 31, 45],       # Количество листьев в дереве. Главный параметр контроля сложности.
    'max_depth': [-1, 10, 15],        # Максимальная глубина дерева. -1 означает неограниченную (контролируется num_leaves).
    'reg_alpha': [0, 0.1],        # L1 регуляризация
    'reg_lambda': [0, 0.1],       # L2 регуляризация
    'colsample_bytree': [0.7, 0.8], # Доля признаков для каждого дерева
    'subsample': [0.7, 0.8],      # Доля строк для каждого дерева (bagging)
    'class_weight': ['balanced']       # Для учета дисбаланса классов
}

print(f"\nСетка параметров для подбора: {param_grid}")


In [ ]:
# --- 4. Инициализация LGBMClassifier и GridSearchCV ---
print("\nНачинаем подбор гиперпараметров LightGBM с помощью GridSearchCV на подвыборке...")

# Используем 'train_data' для передачи подвыборки, чтобы early_stopping_rounds работал корректно.
# eval_set нужен для early_stopping_rounds.
# `n_jobs=-1` для ускорения процесса кросс-валидации.
lgbm_classifier = LGBMClassifier(objective='binary',
                                 metric='auc',
                                 random_state=42,
                                 n_jobs=-1,
                                 # `early_stopping_round` задается при вызове .fit(), а не в конструкторе.
                                 # `verbose=-1` отключает вывод прогресса для каждого дерева,
                                 # но GridSearchCV сам управляет verbosity.
                                 verbose=-1)

# Определяем параметры для GridSearchCV
grid_search = GridSearchCV(estimator=lgbm_classifier,
                           param_grid=param_grid,
                           cv=3,
                           scoring='roc_auc',
                           verbose=3,
                           n_jobs=-1)

start_time = time.time()

# Запускаем поиск по сетке на ПОДВЫБОРКЕ данных
# Для корректной работы early_stopping_rounds, обычно нужно передать X_train_subset
# и y_train_subset в eval_set, но GridSearchCV сам разбивает данные на фолды.
# Так что просто передаем subset.
grid_search.fit(X_train_subset, y_train_subset)

end_time = time.time()
tuning_time = end_time - start_time

print(f"\nПодбор гиперпараметров LightGBM завершен. Время подбора: {tuning_time:.2f} секунд.")


In [ ]:
# --- 5. Обучение финальной модели на всем X_train ---
print("\nОбучаем финальную модель LightGBM на ВСЕМ обучающем наборе...")
best_params = grid_search.best_params_
print(f"Лучшие найденные параметры: {best_params}")
print(f"Лучший ROC AUC на кросс-валидации (по подвыборке): {grid_search.best_score_:.4f}")

final_lgbm_model = LGBMClassifier(objective='binary',
                                  metric='auc',
                                  random_state=42,
                                  n_jobs=-1,
                                  verbose=-1, # Отключаем verbose для финального обучения
                                  **best_params)

# Обучение финальной модели.
# В идеале, нужно выделить часть X_train как validation set для early_stopping.
# Для демонстрации, просто обучим с заданным n_estimators.
# Если в best_params есть n_estimators, оно будет использовано.
# Если бы мы хотели использовать early_stopping, нам нужен был бы eval_set.
# Например:
# X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)
# final_lgbm_model.fit(X_train_part, y_train_part,
#                      eval_set=[(X_val_part, y_val_part)],
#                      eval_metric='auc',
#                      callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]) # verbose=False, чтобы не спамить лог

start_time_final_train = time.time()
# Обучаем финальную модель на всем X_train (который уже float32)
final_lgbm_model.fit(X_train, y_train)
end_time_final_train = time.time()
final_training_time = end_time_final_train - start_time_final_train

print(f"Финальная модель LightGBM обучена на полном наборе данных. Время обучения: {final_training_time:.2f} секунд.")


In [ ]:
# --- 6. Прогнозирование на тестовой выборке (с лучшей моделью) ---
print("\nВыполняем предсказания на тестовой выборке...")
y_pred_proba = final_lgbm_model.predict_proba(X_test)[:, 1]
y_pred_default_threshold = final_lgbm_model.predict(X_test) # Предсказания с порогом 0.5

# --- 7. Построение ROC AUC кривой ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(9, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve (LightGBM - Final Model)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"Значение ROC AUC на тестовой выборке: {roc_auc:.4f}")


In [ ]:
# --- 8. Построение матрицы ошибок (Confusion Matrix) для порога по умолчанию (0.5) ---
print("\n--- Оценка с порогом классификации по умолчанию (0.5) ---")
cm_default = confusion_matrix(y_test, y_pred_default_threshold)
disp_default = ConfusionMatrixDisplay(confusion_matrix=cm_default, display_labels=final_lgbm_model.classes_)

plt.figure(figsize=(8, 8))
disp_default.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title('Матрица ошибок (Confusion Matrix) - Порог 0.5')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.grid(False)
plt.show()

print("\nМатрица ошибок (Confusion Matrix) при пороге 0.5:\n", cm_default)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_default_threshold, average='binary', pos_label=1)
print(f"Precision (для класса 1) при пороге 0.5: {precision:.4f}")
print(f"Recall (для класса 1) при пороге 0.5: {recall:.4f}")
print(f"F1-score (для класса 1) при пороге 0.5: {fscore:.4f}")

In [ ]:
#ПОСТРОИМ МОДЕЛЬ XGBoost

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
from sklearn.model_selection import train_test_split, GridSearchCV
# Импортируем XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
# Разделение данных на признаки (X) и целевую переменную (y)
features_to_drop = ['target']
if 'id' in train_data.columns:
    features_to_drop.append('id')

X = train_data.drop(columns=features_to_drop)
y = train_data['target']

# --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ 1: Приведение всех признаков к float32 ---
print(f"Исходный тип данных признаков: {X.dtypes.iloc[0]}")
X = X.astype(np.float32)
print(f"Новый тип данных признаков: {X.dtypes.iloc[0]} (уменьшено для экономии памяти)")

print(f"Размер X (признаков): {X.shape}")
print(f"Размер y (целевой переменной): {y.shape}")

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}, y_test: {y_test.shape}")


# --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ 2: Создание подвыборки для GridSearchCV ---
subset_ratio = 0.1 # Использовать 10% обучающих данных для Grid Search

print(f"\nСоздаем подвыборку ({subset_ratio*100:.0f}%) обучающих данных для Grid Search...")
X_train_subset, _, y_train_subset, _ = train_test_split(
    X_train, y_train, test_size=(1-subset_ratio), random_state=42, stratify=y_train
)
print(f"Размер X_train_subset для Grid Search: {X_train_subset.shape}")

In [ ]:
# --- 3. Определение сетки для подбора гиперпараметров XGBoost ---
# Пример сетки. Может потребоваться более тонкая настройка.
param_grid = {
    'n_estimators': [300, 500], # Уменьшаем для скорости подбора
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5], # Уменьшаем глубину
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [0, 0.1],
    # Расчет scale_pos_weight: count(negative_class) / count(positive_class)
    'scale_pos_weight': [20] # Примерное значение, подберите более точно под свои данные
}

print(f"\nСетка параметров для подбора: {param_grid}")

In [ ]:
# --- 4. Инициализация XGBClassifier и GridSearchCV ---
print("\nНачинаем подбор гиперпараметров XGBoost с помощью GridSearchCV на подвыборке...")

# `use_label_encoder=False` для подавления предупреждений.
# `eval_metric='auc'` для оценки.
xgb_classifier = XGBClassifier(objective='binary:logistic',
                               eval_metric='auc',
                               use_label_encoder=False,
                               random_state=42,
                               n_jobs=-1) # Использовать все ядра CPU

grid_search = GridSearchCV(estimator=xgb_classifier,
                           param_grid=param_grid,
                           cv=3,
                           scoring='roc_auc',
                           verbose=3,
                           n_jobs=-1)

start_time = time.time()
grid_search.fit(X_train_subset, y_train_subset) # Подбор на подвыборке
end_time = time.time()
tuning_time = end_time - start_time

print(f"\nПодбор гиперпараметров XGBoost завершен. Время подбора: {tuning_time:.2f} секунд.")# --- 4. Инициализация XGBClassifier и GridSearchCV ---


In [ ]:
# --- 5. Обучение финальной модели на всем X_train с early_stopping_round ---
print("\nОбучаем финальную модель XGBoost на ВСЕМ обучающем наборе с early_stopping_round...")

best_params = grid_search.best_params_
print(f"Лучшие найденные параметры: {best_params}")
print(f"Лучший ROC AUC на кросс-валидации (по подвыборке): {grid_search.best_score_:.4f}")

# --- Разделяем X_train на обучающую и валидационную части ---
# Для ранней остановки обучения финальной модели
X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train # 10% данных для валидации
)

print(f"\nРазделяем X_train на обучающую ({X_train_part.shape[0]}) и валидационную ({X_val_part.shape[0]}) части для early_stopping.")


In [ ]:
# Инициализация финальной модели с найденными параметрами
final_xgb_model = XGBClassifier(objective='binary:logistic',
                                eval_metric='auc',
                                use_label_encoder=False,
                                random_state=42,
                                n_jobs=-1,
                                # Передаем early_stopping_rounds как параметр модели
                                early_stopping_rounds=100,
                                **best_params)

start_time_final_train = time.time()

# Обучение финальной модели
# eval_set используется для ранней остановки
final_xgb_model.fit(X_train_part, y_train_part,
                    eval_set=[(X_val_part, y_val_part)],
                    verbose=False) # verbose=False, чтобы не выводить прогресс для каждого шага

end_time_final_train = time.time()
final_training_time = end_time_final_train - start_time_final_train

print(f"Финальная модель XGBoost обучена. Время обучения: {final_training_time:.2f} секунд.")
# Получаем количество деревьев, на котором остановилась модель
print(f"Обучено деревьев: {final_xgb_model.best_iteration}")

In [ ]:
# --- 6. Прогнозирование на тестовой выборке (с лучшей моделью) ---
print("\nВыполняем предсказания на тестовой выборке...")
y_pred_proba = final_xgb_model.predict_proba(X_test)[:, 1]
# Делаем предсказания с порогом 0.5 (чтобы получить классы для Confusion Matrix)
y_pred_default_threshold = final_xgb_model.predict(X_test)


In [ ]:
# --- 7. Построение ROC AUC кривой ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(9, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve (XGBoost - Final Model)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"Значение ROC AUC на тестовой выборке: {roc_auc:.4f}")

In [ ]:
# --- 8. Построение матрицы ошибок (Confusion Matrix) для порога по умолчанию (0.5) ---
print("\n--- Оценка с порогом классификации по умолчанию (0.5) ---")
cm_default = confusion_matrix(y_test, y_pred_default_threshold)
# Получаем метки классов из модели, если они были определены (обычно это 0 и 1)
# final_xgb_model.classes_ может быть недоступен, если он не установлен явно
# Но для бинарной классификации с 0 и 1 это обычно работает
try:
    display_labels = final_xgb_model.classes_
except AttributeError:
    display_labels = [0, 1] # Если classes_ недоступен, используем стандартные метки

disp_default = ConfusionMatrixDisplay(confusion_matrix=cm_default, display_labels=display_labels)

plt.figure(figsize=(8, 8))
disp_default.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title('Матрица ошибок (Confusion Matrix) - Порог 0.5')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.grid(False)
plt.show()

print("\nМатрица ошибок (Confusion Matrix) при пороге 0.5:\n", cm_default)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_default_threshold, average='binary', pos_label=1)
print(f"Precision (для класса 1) при пороге 0.5: {precision:.4f}")
print(f"Recall (для класса 1) при пороге 0.5: {recall:.4f}")
print(f"F1-score (для класса 1) при пороге 0.5: {fscore:.4f}")

Вывод:В ходе проделанной работы произведено итеративное чтение большого объёма данных, собран итоговый датафрейм, состоящий из признаков для обучения модели. После обучения модели, были получены следующие метрики качества:
1) Модель CatBoost: ROC AUC = 0.7559
2) Модель CatBoost (подбор по сетке): ROC AUC = 0.759
3) Модель LogisticRegression: ROC AUC = 0.7314
4) Модель LightGBM: ROC AUC = 0.7604
5) Модель XGBoost: ROC AUC = 0.7557